## Lee TH., Seregina E.: "Combining Forecasts under Structural Breaks Using Graphical LASSO"

### This Python notebook can be used to reproduce Monte Carlo results for MSFE in Supplementary Appendix Figures E2, E4,  E7, E9

In [ ]:
%pip install regain==0.3.9
# %pip install numpy
# %pip install scipy
# %pip install scikit-learn
# %pip install pandas

#### Please make sure to place "GL.py" and "TVGL.py" in the same directory as this notebook

In [ ]:
from __future__ import division
from GL import GraphicalLasso
# from TVGL2 import TimeGraphicalLasso
from TVGL import TimeGraphicalLasso
from regain.datasets import make_dataset
from regain.utils import error_norm_time
import numpy as np 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy
import scipy.linalg   # SciPy Linear Algebra Library
import pandas as pd
import statistics
import math
from scipy.linalg import cholesky
import sklearn
from sklearn import datasets
from numpy.linalg import inv
from sklearn.covariance import GraphicalLassoCV
import warnings
warnings.filterwarnings("ignore")
from numpy import savetxt

In [ ]:
##############FUNCTIONS#############
def portfolios(X,estsigm):
    X = X.to_numpy()
    mu = np.mean(X,axis=0).reshape(X.shape[1],1)
    p = len(mu) 
    one = np.ones([p,1])
    phi = one.T@estsigm@one
    #GMV##
    gmv = (estsigm @ one) / phi 
    return [gmv]  
###################################################################

def MakeAR(y,lag):
    x = np.ones((y.shape[0]-lag, 1))
    if lag !=(-1):
        for l in range(0,lag+1):
            x=np.concatenate([x,y[(lag-l):(len(y) - l)]], axis=1)
        else:
            x=x
    return [x]   
####################################################################
def CVlasso(penalty, X,Y, y2, k1, q, window, forecasters, truers, Fhat, lmb, alpha_set, beta_set): #X is returns, Y is residuals, K=k,
    inX = X[0:(window-q)]
    inY = Y[0:(window-q)]
    iny2 = y2[0:(window-q)]
    #################SFEs ARE HERE##################
    meanSFE=np.zeros([len(alpha_set),len(beta_set)])
    for alpha in range(0,len(alpha_set)):
        for beta in range(0,len(beta_set)):
            if penalty == '':
                tvfgl = TimeGraphicalLasso(max_iter=100, alpha = alpha_set[alpha], beta = beta_set[beta]).fit(inY, iny2)
            else:    
                tvfgl = TimeGraphicalLasso(psi=penalty, max_iter=100, alpha = alpha_set[alpha], beta = beta_set[beta]).fit(inY, iny2)
            if k1==1:
                Thetatvfgl=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                Thetatvfgl=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]

            portfolio_tvfgl = portfolios(inX, Thetatvfgl) #IMPORTANT: use non-standardized returns when computing portfolio weights!!!
            weight_global = portfolio_tvfgl[0].T
            competing = forecasters[(window-q):]
            true_y = truers.iloc[(window-q):]
            # competing = competing.to_numpy()
            SFE = []
            w1 = weight_global.T
            for kappa in range(competing.shape[0]):
                FE=w1.T@competing.iloc[kappa,:]
        #################SFEs ARE HERE##################
                SFE1=(true_y.iloc[kappa,:] - FE)**2
                SFE.append(SFE1)

            meanSFE[alpha,beta] = np.mean(SFE)    
        same = []
        meanSFE_0 = []
    for d in range(0,(len(iny2)-1)):
        if iny2[d] ==iny2[d+1]:
            same = 0
        else:
            same = 1
        if same == 1:     
            [alphaopt1, betaopt1] = [alpha_set[np.where(meanSFE == np.nanmin(meanSFE))[0]], beta_set[np.where(meanSFE == np.nanmin(meanSFE))[1]]]
        else:
            meanSFE_0 = meanSFE[:,1]
        # betaopt1 = np.asarray(betaopt1)
            [alphaopt1, betaopt1] = [alpha_set[np.where(meanSFE_0 == np.nanmin(meanSFE_0))[0]], np.ravel(np.asarray(beta_set[np.where(meanSFE == np.nanmin(meanSFE))[1]][0]))]
        return np.array([alphaopt1,betaopt1])
    

def CV_gamma(gamma_set,r, r1_cv, r2_cv, k):          
    err_gamma=np.zeros([len(gamma_set),1])
    err_loo=np.zeros([r2_cv.shape[0],1])    
    for gamma in range(0,len(gamma_set)):
        for loo in range(0,r2_cv.shape[0]):
            # r = F@Lambda[0:kDGP,] + eps
            r1_upd = r1_cv*gamma_set[gamma]
            r2_upd = np.delete(r2_cv, loo, axis=0)
            ##leaving one time -series out
            r_cv = np.concatenate((r1_upd, r2_upd), axis=0)

            ##estimating factors and loadings with LOO returns           
            L, V = np.linalg.eigh(np.dot(r_cv.T, r_cv))
            idx = L.argsort()[::-1]
            L = L[idx]  # eigenvalues, Nx1
            V = V[:, idx]  # eigenvectors columns, NxN
            lmb = V[:, 0:k]  # kx1
            Fhat = np.dot(r_cv, lmb)  # Txr (r=1 for PC1)
            sum_i = np.zeros([r_cv.shape[1],1])
            for i in range(r.shape[1]):
                sum_j = np.zeros([r2_cv.shape[0],1])
                ##computing sum of squared errors for the post-break period
                for j in range(0,r2_cv.shape[0]):
                    sum_j[j] = (r2_cv[j,i]-Fhat[j,:]@lmb.T[:,i])**2
                sum_i[i] = np.sum(sum_j) 
            ##collecting SSEs for all LOOs    
            err_loo[loo] = np.sum(sum_i)/(r.shape[1]*(r2_cv.shape[0]))   
        ##compute average of all LOOs for one gamma                
        err_gamma[gamma] = np.sum(err_loo)
    return  gamma_set[np.where(err_gamma == np.min(err_gamma))[0]]

In [ ]:
########PARAMETERS################################################
gamma_opt_zero = 0
gamma_set = np.arange(0,1.05,0.05)  
alpha_set = np.array([0, 0.25, 0.5, 1, 10, 30]).astype(float)
beta_set = np.array([0, 0.25, 0.5, 1, 10, 30]).astype(float)
##################################################################
c1 = 0.75
c2 = 0.9
r = 5 #Factors in DGP
sigmaF = 1
sigmaX = 1
sigmaY = sigmaX
p1 = 8 #Number of AR(l) models
r2 = 3 #Estimated factors. Used to be 2
r3 =5 #Factors in modeling Forecast Errors
s = p1*(1+r2) #Forecasters
rho=0.9 #in regime 1 ## USED TO BE 0.2, in the paper figure 3 says rho=0.9, but it was 0.2
rho2 = 0.8 #in regime 2 ##rho2 is not used for defining regimes, not sure why its here
phi = 0.8
N=100
reps = 500
#####################################################################
# sample_size=range(400,1250,200)
sample_size=range(800,1250,200)
# sample_size=range(1000,1250,200)
####Try
# N=100
# sample_size=range(400,650,200)

#####PLACEHOLDERS
##########Matrices Placeholders######
###for h=1
cum_GL = np.zeros((len(sample_size),1))
cum_FGL = np.zeros((len(sample_size),1))
cum_EW = np.zeros((len(sample_size),1))
cum_TVFGL_laplacian = np.zeros((len(sample_size),1))
cum_TVFGL_l1 = np.zeros((len(sample_size),1))
cum_TVFGL_grouplasso = np.zeros((len(sample_size),1))
cum_TVFGL_laplacian_load = np.zeros((len(sample_size),1))
cum_TVFGL_l1_load = np.zeros((len(sample_size),1))
cum_TVFGL_grouplasso_load = np.zeros((len(sample_size),1))

In [ ]:
#####Appendix Figure E2: BREAK IN BOTH PRECISION AND LOADINGS##################
count=-1
# count=1 #if interrupted and continue to run
for T in sample_size:
    count = count + 1
    m2 = int(round(2*T/3)) #Forecasting observations
    m1=T-m2 
    window = int(round(m2/2))
    T1 = m1 + int(round(window/2))
    ###for h=1
    error_EW =  np.zeros((reps+1,1))
    error_GL =  np.zeros((reps+1,1))
    error_FGL =  np.zeros((reps+1,1))
    error_TVFGL_laplacian =  np.zeros((reps+1,1))
    error_TVFGL_l1 =  np.zeros((reps+1,1))
    error_TVFGL_grouplasso =  np.zeros((reps+1,1))
    error_TVFGL_laplacian_load =  np.zeros((reps+1,1))
    error_TVFGL_l1_load =  np.zeros((reps+1,1))
    error_TVFGL_grouplasso_load =  np.zeros((reps+1,1))
    
    
    ###Incorporating break in theta through break in c2####
   
    #######################################################
    for z in range(reps+1):  
        e = np.random.randn(T,1)      
        theta = np.zeros((T,1))  
        breaks_full = np.zeros((T,1))
        for w in range(T):
            if w <= T1:
                c2 = 0.3  
                theta[w] = ((w+1)**c1)*(c2**(w))
                breaks_full[w,]=0
            else:
                c2 = 0.4  ###for normal break was 0.9
                theta[w] = ((w+1)**c1)*(c2**(w))
                breaks_full[w,]=1
        breaks_full = breaks_full.astype(int)
        breaks_full = np.ravel(breaks_full)
        

        #######################################################
        y1 = np.zeros((T,1)) 
        for g in range(T):
            y1[g] = theta[0:(g+1)].T@np.flip(e[0:(g+1)])
         
        
        ###################################################
        #creating Toeplitz matrix and its Cholesky decomposition
        ####For regime 1
        O = np.zeros((N, N))
        np.fill_diagonal(O, np.ones(N))     
        for h in range(1,N):
            np.fill_diagonal(O[h:], rho**np.repeat(h, N-h))
            np.fill_diagonal(O[:,h:], rho**np.repeat(h, N-h))
        
        Lambda = cholesky(O, lower=False)
        
        ######################################
        F = np.zeros((T,r))
        F[0,] = sigmaF*np.random.randn(1,r)
        v=np.random.randn(T,r)
        for j in range(0,r):
            for t in range(1,T):
                F[t,j] = phi*F[t-1,j]+sigmaF*v[t,j]
                
        ##################################################        
        ###############NO BREAKS##########################
        X = F@Lambda[0:r,] + sigmaX*np.random.randn(T,N)
        ##################################################
        alpha = np.random.randn(r,1)+1
        y2 = np.zeros((T,1))
        y2[1:T]= F[0:(T-1),]@alpha +0*sigmaY*np.random.randn((T-1),1)
        y=y1+y2
        
        Yhat = np.zeros((m2,s))
       
        
        for col in range(0,p1):  # Col=1 when no lags
            for k in range(0,r2+1): # Number of factors
                for row in range(0,m2):
                    xx = MakeAR(y[(row):(m1-1+row)], (col-1))[0]
                    if col==0 and k!=0:
                        xx = np.ones(((m1-1),1))

                    if k==0:
                        xx=xx
                        coef = np.linalg.inv(xx.T@xx)@xx.T@y[(row+col):(m1+row)]
                    else:
                        data = X[(row):(m1-1+row),]
                        L, V = np.linalg.eigh(np.dot(data.T, data))
                        idx = L.argsort()[::-1]
                        L = L[idx]  # eigenvalues, Nx1
                        V = V[:, idx]  # eigenvectors columns, NxN
                        lmb = V[:, 0:k]  # kx1
                        Fhat = np.dot(data, lmb)
                        if col==0:
                            Fhat = Fhat[(col):(m1 -1),]
                            xx = np.concatenate([xx,Fhat], axis=1)
                        else:
                            Fhat = Fhat[(col-1):(m1 -1),]
                            xx = np.concatenate([xx,Fhat], axis=1)

                        if col==0:
                            coef = np.linalg.inv(xx.T@xx)@xx.T@y[(row+col+1):(m1 +row)]
                        else:
                            coef = np.linalg.inv(xx.T@xx)@xx.T@y[(row+col):(m1 +row)]   
                    if col == 0: # 0 lags 
                        if k==0:
                            Yhat[row,((r2+1)*(col)+k)] = coef #if col=1 and k=0
                        else:
                            Yhat[row,((r2+1)*(col)+k)] = np.concatenate([np.array([1]), Fhat[Fhat.shape[0]-1,]], axis=0)@coef #if col=0 and k>0          
                    else:         # >=1 lags
                        if k==0:
                            Yhat[row,((r2+1)*(col)+k)] = np.concatenate([np.array([1]),np.ravel(np.flip(y[(m1 +row-col+1):(m1 +row+1)]))], axis=0)@coef                    
                            # Yhat[row,((r2+1)*(col)+k)] = np.array([1,np.flip(y[(m1 +row-col+1):(m1 +row+1)])])@coef #if col>1 and k=0
                        else:
                            Yhat[row,((r2+1)*(col)+k)] = np.concatenate([np.array([1]),np.ravel(np.flip(y[(m1 +row-col+1):(m1 +row+1)])), Fhat[Fhat.shape[0]-1,]], axis=0)@coef 
                            # USED TO BE np.ravel(np.flip(y[(m1 +row-col+1):(m1 +row+1)])) PLUS instead of MINUS for some reason
 ##################################################################################333
        
        forERR=np.full((Yhat.shape[0], Yhat.shape[1]), y[(m1):T]) - Yhat
               
        y =  pd.DataFrame(y)
        ###for h=1
        trying = forERR
        Yhat = pd.DataFrame(Yhat)
        forERR = pd.DataFrame(forERR)
        y_oos = pd.DataFrame(y[(m1):T])
        breaks = breaks_full[(m1):T]
        
        ###############################################################
        ####FORECAST ERRORS MATRICES####
        ###for h=1
        FEEW= np.zeros((m2-window,1))
        FEGL= np.zeros((m2-window,1))
        FEFGL= np.zeros((m2-window,1))
        FETVFGL_laplacian= np.zeros((m2-window,1))
        FETVFGL_l1= np.zeros((m2-window,1))
        FETVFGL_grouplasso= np.zeros((m2-window,1))
        
        FETVFGL_laplacian_load= np.zeros((m2-window,1))
        FETVFGL_l1_load= np.zeros((m2-window,1))
        FETVFGL_grouplasso_load= np.zeros((m2-window,1))
        
        #####SFEs####
        SFEEW= np.zeros((m2-window,1))
        SFEGL= np.zeros((m2-window,1))
        SFEFGL= np.zeros((m2-window,1))        
        SFETVFGL_laplacian= np.zeros((m2-window,1))
        SFETVFGL_l1= np.zeros((m2-window,1))
        SFETVFGL_grouplasso= np.zeros((m2-window,1))
        
        SFETVFGL_laplacian_load= np.zeros((m2-window,1))
        SFETVFGL_l1_load= np.zeros((m2-window,1))
        SFETVFGL_grouplasso_load= np.zeros((m2-window,1))
        
        for j in range(0, m2-window):
            print('T =', T, 'z =', z, 'out of', reps, 'j =', j, 'out of', m2-window)
            ###for h=1
            set1 = pd.DataFrame(trying)
            set1 = set1.iloc[j:(j+window),:]
            ###these r1 and r2 below are for time-varying
            err1 = pd.DataFrame(trying[0:T1,])
            err2 = pd.DataFrame(trying[T1:,])
            #####for time-varying######
            forecasters = Yhat.iloc[j:(j+window),:]
            truers = y_oos.iloc[j:(j+window),:]
            y2 = breaks[j:(j+window),].astype(int)
            y2 = y2.astype(int)
            
            #############################
            ###Estimating factors and loadings here
            k1=r2
            ###for h=1    
            ###the part below w/ gamma_opt should be valid for all h's
            if (j+window) > T1 and j <= T1:
                r1_cv = err1.iloc[j:T1,:]
                r2_cv = err2.iloc[0:(j+window-T1),:]
                r1_cv = r1_cv.to_numpy()
                r2_cv = r2_cv.to_numpy()
                gamma_opt = CV_gamma(gamma_set,set1, r1_cv, r2_cv, k)
            else:
                gamma_opt = 1
            print('gamma_opt =', gamma_opt)
           
            set2 = set1.to_numpy()
            set2 = StandardScaler().fit(set2).transform(set2)
            ####modified returns for time-varying loadings only!!!
            set2_load = set2.copy()
            for row in range(set2_load.shape[0]):
                for col in range(set2_load.shape[1]):
                    if row < (T1-j) and (j+window) > T1 and j <= T1:
                        set2_load[row,col]=gamma_opt* set2_load[row,col] 

            L_load, V_load = np.linalg.eigh(np.dot(set2_load.T, set2_load))
            idx_load = L_load.argsort()[::-1]
            L_load = L_load[idx_load]  # eigenvalues, Nx1
            V_load = V_load[:, idx_load]  # eigenvectors columns, NxN
            lmb_load = V_load[:, 0:k1]  # kx1
      
            ###According to Su (2017, JoE) if we obtain Fhat
            ###as usual they are only consistent for a rotational version
            ###hence, to get a consistent estimator use a two-stage procedure (OLS)
            Fhat_load = set2@lmb_load@np.linalg.inv(lmb_load.T@lmb_load)
            Y_load = set1 - Fhat_load@lmb_load.T ##these are the residuals
            
            covariate_load = Fhat_load
            betas_load = lmb_load.T
            
            ###################using only post-break data (gamma= 0, assuming that there are at least 30 post-breaks observations)
            set2_load_zero = set2.copy()
            for row in range(set2_load_zero.shape[0]):
                for col in range(set2_load_zero.shape[1]):
                    if row < (T1-j) and (j+window-T1) > 30 and j <= T1:
                        set2_load_zero[row,col]=gamma_opt_zero* set2_load_zero[row,col] 

            L_load_zero, V_load_zero = np.linalg.eigh(np.dot(set2_load_zero.T, set2_load_zero))
            idx_load_zero = L_load_zero.argsort()[::-1]
            L_load_zero = L_load_zero[idx_load_zero]  # eigenvalues, Nx1
            V_load_zero = V_load_zero[:, idx_load_zero]  # eigenvectors columns, NxN
            lmb_load_zero = V_load_zero[:, 0:k1]  # kx1
      
            ###According to Su (2017, JoE) if we obtain Fhat
            ###as usual they are only consistent for a rotational version
            ###hence, to get a consistent estimator use a two-stage procedure (OLS)
            Fhat_load_zero = set2@lmb_load_zero@np.linalg.inv(lmb_load_zero.T@lmb_load_zero)
            Y_load_zero = set1 - Fhat_load_zero@lmb_load_zero.T ##these are the residuals
            
            covariate_load_zero = Fhat_load_zero
            betas_load_zero = lmb_load_zero.T
            ###############################################################################
            ##########NOT time-varying for h=1 here
            ###############################################################################
            L, V = np.linalg.eigh(np.dot(set2.T, set2))
            idx = L.argsort()[::-1]
            L = L[idx]  # eigenvalues, Nx1
            V = V[:, idx]  # eigenvectors columns, NxN
            lmb = V[:, 0:k1]  # kx1
            Fhat = np.dot(set2, lmb)  # Txr (r=1 for PC1)
            Y = set1 - Fhat@lmb.T 
            Y = Y.to_numpy()
            
            covariate = Fhat
            betas = lmb.T
            
 ###############################################################################################################################                    
            ###for h=1
            ######## GLASSO###########
            GL = GraphicalLassoCV().fit(set1)
            theta_GL = GL.get_precision()
            
            ########Factor GLASSO#######
            FGL = GraphicalLassoCV().fit(Y)
            theta_FGL_error = FGL.get_precision()
            
            if k1==1:
                theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            else:
                theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            
            ########Time-Varying TVFGL(only precision time-varying i.e. gamma=1)#########   
            tuning_laplacian = CVlasso(penalty = '', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)##laplacian (ridge) is defauls and was used at the beginning
            tuning_l1 = CVlasso(penalty = 'l1', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)
            tuning_grouplasso = CVlasso(penalty = 'l2', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)                       
            ###laplacian       
            tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
     
            ###l1       
            tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
         ###############################################################   
            ###group lasso     
            tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
         
            print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)
        ###############################################################   
    ############################################################### 
    #########Time-Varying TVFGL(BOTH precision and loadings time-varying)#########        
    ###############################################################
            tuning_laplacian_load = CVlasso(penalty = '', X=set1,Y=Y_load,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat_load, lmb=lmb_load, alpha_set=alpha_set, beta_set=beta_set)##laplacian (ridge) is defauls and was used at the beginning
            tuning_l1_load = CVlasso(penalty = 'l1', X=set1,Y=Y_load,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat_load, lmb=lmb_load, alpha_set=alpha_set, beta_set=beta_set)
            tuning_grouplasso_load = CVlasso(penalty = 'l2', X=set1,Y=Y_load,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat_load, lmb=lmb_load, alpha_set=alpha_set, beta_set=beta_set)
            
            print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)
            ###laplacian      
            tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian_load[0][0], beta = tuning_laplacian_load[1][0]).fit(Y_load, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_laplacian_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_laplacian_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]

            ###l1  
            ##sometimes tuning for l1 won't converge so added these:
            if len(tuning_l1_load[0]) == 0:
                alpha_l1_load = 0
            else:
                alpha_l1_load = tuning_l1_load[0][0]

            if len(tuning_l1_load[1]) == 0:
                beta_l1_load = 0
            else:
                beta_l1_load = tuning_l1_load[1][0]
            ##################################################
            tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = alpha_l1_load, beta = beta_l1_load).fit(Y_load, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_l1_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_l1_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
         ###############################################################   
            ###group lasso     
            tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso_load[0][0], beta = tuning_grouplasso_load[1][0]).fit(Y_load, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_grouplasso_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_grouplasso_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
         ###############################################################   
           ###########Weights###################
            weight_EW = np.repeat(1/s,s)
            weight_GL = portfolios(set1,theta_GL)[0]
            weight_FGL = portfolios(set1,theta_FGL)[0]       
            weight_TVFGL_laplacian = portfolios(set1,theta_TVFGL_laplacian )[0]
            weight_TVFGL_l1 = portfolios(set1,theta_TVFGL_l1)[0]
            weight_TVFGL_grouplasso = portfolios(set1,theta_TVFGL_grouplasso)[0]
            weight_TVFGL_laplacian_load = portfolios(set1,theta_TVFGL_laplacian_load )[0]
            weight_TVFGL_l1_load = portfolios(set1,theta_TVFGL_l1_load)[0]
            weight_TVFGL_grouplasso_load = portfolios(set1,theta_TVFGL_grouplasso_load)[0]
            
                      
            #########################################################################################################
           #######MODEL FORECASTS ARE HERE######
            FEEW[j]=y_oos.iloc[window +j,:] - weight_EW.T@Yhat.iloc[window +j,:]
            FEGL[j]=y_oos.iloc[window +j,:] - weight_GL.T@Yhat.iloc[window +j,:]
            FEFGL[j]=y_oos.iloc[window +j,:] - weight_FGL.T@Yhat.iloc[window +j,:]
            FETVFGL_laplacian[j]=y_oos.iloc[window +j,:] - weight_TVFGL_laplacian.T@Yhat.iloc[window +j,:] 
            FETVFGL_l1[j]=y_oos.iloc[window +j,:] - weight_TVFGL_l1.T@Yhat.iloc[window +j,:] 
            FETVFGL_grouplasso[j]=y_oos.iloc[window +j,:] - weight_TVFGL_grouplasso.T@Yhat.iloc[window +j,:] 
            
            FETVFGL_laplacian_load[j]=y_oos.iloc[window +j,:] - weight_TVFGL_laplacian_load.T@Yhat.iloc[window +j,:] 
            FETVFGL_l1_load[j]=y_oos.iloc[window +j,:] - weight_TVFGL_l1_load.T@Yhat.iloc[window +j,:] 
            FETVFGL_grouplasso_load[j]=y_oos.iloc[window +j,:] - weight_TVFGL_grouplasso_load.T@Yhat.iloc[window +j,:]
            
            #################SFEs ARE HERE##################
            ###for h=1
            SFEEW[j]=(FEEW[j])**2
            SFEGL[j]=(FEGL[j])**2
            SFEFGL[j]=(FEFGL[j])**2
            SFETVFGL_laplacian[j]=(FETVFGL_laplacian[j])**2
            SFETVFGL_l1[j]=(FETVFGL_l1[j])**2
            SFETVFGL_grouplasso[j]=(FETVFGL_grouplasso[j])**2
            SFETVFGL_laplacian_load[j]=(FETVFGL_laplacian_load[j])**2
            SFETVFGL_l1_load[j]=(FETVFGL_l1_load[j])**2
            SFETVFGL_grouplasso_load[j]=(FETVFGL_grouplasso_load[j])**2

            ####################   
        ###for h=1
        MSFE_EW = np.mean(SFEEW)
        MSFE_GL = np.mean(SFEGL)
        MSFE_FGL = np.mean(SFEFGL)
        MSFE_TVFGL_laplacian = np.mean(SFETVFGL_laplacian)
        MSFE_TVFGL_l1 = np.mean(SFETVFGL_l1)
        MSFE_TVFGL_grouplasso = np.mean(SFETVFGL_grouplasso)
        MSFE_TVFGL_laplacian_load = np.mean(SFETVFGL_laplacian_load)
        MSFE_TVFGL_l1_load = np.mean(SFETVFGL_l1_load)
        MSFE_TVFGL_grouplasso_load = np.mean(SFETVFGL_grouplasso_load)
       
        print('MSFE_EW =', MSFE_EW, 'MSFE_GL =', MSFE_GL, 'MSFE_FGL =', MSFE_FGL,'MSFE_TVFGL_laplacian =', MSFE_TVFGL_laplacian,
             'MSFE_TVFGL_l1 =', MSFE_TVFGL_l1, 'MSFE_TVFGL_grouplasso =', MSFE_TVFGL_grouplasso,
             'MSFE_TVFGL_laplacian_load =', MSFE_TVFGL_laplacian_load,
             'MSFE_TVFGL_l1_load =', MSFE_TVFGL_l1_load, 'MSFE_TVFGL_grouplasso_load =', MSFE_TVFGL_grouplasso_load)

        ##########################################################
        ###for h=1
        error_EW[z] = MSFE_EW
        error_GL[z] = MSFE_GL
        error_FGL[z] = MSFE_FGL
        error_TVFGL_laplacian[z] = MSFE_TVFGL_laplacian
        error_TVFGL_l1[z] = MSFE_TVFGL_l1
        error_TVFGL_grouplasso[z] = MSFE_TVFGL_grouplasso
        
        error_TVFGL_laplacian_load[z] = MSFE_TVFGL_laplacian_load
        error_TVFGL_l1_load[z] = MSFE_TVFGL_l1_load
        error_TVFGL_grouplasso_load[z] = MSFE_TVFGL_grouplasso_load

       
    #####################  
    
    ###for h=1
    cum_EW[count] = np.mean(error_EW)
    cum_GL[count] = np.mean(error_GL)
    cum_FGL[count] = np.mean(error_FGL)
    cum_TVFGL_laplacian[count] = np.mean(error_TVFGL_laplacian)
    cum_TVFGL_l1[count] = np.mean(error_TVFGL_l1)
    cum_TVFGL_grouplasso[count] = np.mean(error_TVFGL_grouplasso)
    
    cum_TVFGL_laplacian_load[count] = np.mean(error_TVFGL_laplacian_load)
    cum_TVFGL_l1_load[count] = np.mean(error_TVFGL_l1_load)
    cum_TVFGL_grouplasso_load[count] = np.mean(error_TVFGL_grouplasso_load)
    
       
cum_h1 = np.concatenate((cum_EW,cum_GL,cum_FGL,cum_TVFGL_laplacian, cum_TVFGL_l1,cum_TVFGL_grouplasso,cum_TVFGL_laplacian_load,
                         cum_TVFGL_l1_load,cum_TVFGL_grouplasso_load), axis=1)
    
# savetxt('cum_h1_smallbreak_c1is075_cont.csv', cum_h1, delimiter=',') #before IJF revision
savetxt('cum_h1_rhois09_c1is075_cont.csv', cum_h1, delimiter=',')


In [ ]:
#####Appendix Figure E4: NO BREAK##################
count=-1
for T in sample_size:
    print('T =', T)
    count = count + 1
    m2 = int(round(2*T/3)) #Forecasting observations
    m1=T-m2 
    window = int(round(m2/2))
#     T1 = m1 + int(round(window/2))
    ###for h=1
    error_EW =  np.zeros((reps+1,1))
    error_GL =  np.zeros((reps+1,1))
    error_FGL =  np.zeros((reps+1,1))
    error_TVFGL_laplacian =  np.zeros((reps+1,1))
#     error_TVFGL_l1 =  np.zeros((reps+1,1))
#     error_TVFGL_grouplasso =  np.zeros((reps+1,1))
#     error_TVFGL_max =  np.zeros((reps+1,1))
    
    
    ###Incorporating break in theta through break in c2####
   
    #######################################################
    for z in range(reps+1):      
       ########No break in c2###############################
        e = np.random.randn(T,1)      
        theta = np.zeros((T,1))  
        breaks_full = np.zeros((T,1))
        for w in range(T):
            theta[w] = ((w+1)**c1)*(c2**(w))
        
        #######################################################
        y1 = np.zeros((T,1)) 
        for g in range(T):
            y1[g] = theta[0:(g+1)].T@np.flip(e[0:(g+1)])
                 
        ###################################################
        #creating Toeplitz matrix and its Cholesky decomposition
        ####For regime 1
        O = np.zeros((N, N))
        np.fill_diagonal(O, np.ones(N))     
        for h in range(1,N):
            np.fill_diagonal(O[h:], rho**np.repeat(h, N-h))
            np.fill_diagonal(O[:,h:], rho**np.repeat(h, N-h))
        
        Lambda = cholesky(O, lower=False)
        
        ######################################
        F = np.zeros((T,r))
        F[0,] = sigmaF*np.random.randn(1,r)
        v=np.random.randn(T,r)
        for j in range(0,r):
            for t in range(1,T):
                F[t,j] = phi*F[t-1,j]+sigmaF*v[t,j]
            
        ##################################################        
        ###############NO BREAKS##########################
        X = F@Lambda[0:r,] + sigmaX*np.random.randn(T,N)
        ##################################################
        alpha = np.random.randn(r,1)+1
        y2 = np.zeros((T,1))
        y2[1:T]= F[0:(T-1),]@alpha +0*sigmaY*np.random.randn((T-1),1)
        y=y1+y2
        
        Yhat = np.zeros((m2,s))
       
        
        for col in range(0,p1):  # Col=1 when no lags
            for k in range(0,r2+1): # Number of factors
                for row in range(0,m2):
                    xx = MakeAR(y[(row):(m1-1+row)], (col-1))[0]
                    if col==0 and k!=0:
                        xx = np.ones(((m1-1),1))

                    if k==0:
                        xx=xx
                        coef = np.linalg.inv(xx.T@xx)@xx.T@y[(row+col):(m1+row)]
                    else:
                        data = X[(row):(m1-1+row),]
                        L, V = np.linalg.eigh(np.dot(data.T, data))
                        idx = L.argsort()[::-1]
                        L = L[idx]  # eigenvalues, Nx1
                        V = V[:, idx]  # eigenvectors columns, NxN
                        lmb = V[:, 0:k]  # kx1
                        Fhat = np.dot(data, lmb)
                        if col==0:
                            Fhat = Fhat[(col):(m1 -1),]
                            xx = np.concatenate([xx,Fhat], axis=1)
                        else:
                            Fhat = Fhat[(col-1):(m1 -1),]
                            xx = np.concatenate([xx,Fhat], axis=1)

                        if col==0:
                            coef = np.linalg.inv(xx.T@xx)@xx.T@y[(row+col+1):(m1 +row)]
                        else:
                            coef = np.linalg.inv(xx.T@xx)@xx.T@y[(row+col):(m1 +row)]   
                    if col == 0: # 0 lags 
                        if k==0:
                            Yhat[row,((r2+1)*(col)+k)] = coef #if col=1 and k=0
                        else:
                            Yhat[row,((r2+1)*(col)+k)] = np.concatenate([np.array([1]), Fhat[Fhat.shape[0]-1,]], axis=0)@coef #if col=0 and k>0          
                    else:         # >=1 lags
                        if k==0:
                            Yhat[row,((r2+1)*(col)+k)] = np.concatenate([np.array([1]),np.ravel(np.flip(y[(m1 +row-col+1):(m1 +row+1)]))], axis=0)@coef                    
                            # Yhat[row,((r2+1)*(col)+k)] = np.array([1,np.flip(y[(m1 +row-col+1):(m1 +row+1)])])@coef #if col>1 and k=0
                        else:
                            Yhat[row,((r2+1)*(col)+k)] = np.concatenate([np.array([1]),np.ravel(np.flip(y[(m1 +row-col+1):(m1 +row+1)])), Fhat[Fhat.shape[0]-1,]], axis=0)@coef 
                            # USED TO BE np.ravel(np.flip(y[(m1 +row-col+1):(m1 +row+1)])) PLUS instead of MINUS for some reason
 ##################################################################################333
        
        forERR=np.full((Yhat.shape[0], Yhat.shape[1]), y[(m1):T]) - Yhat
               
        y =  pd.DataFrame(y)
        ###for h=1
        trying = forERR
        Yhat = pd.DataFrame(Yhat)
        forERR = pd.DataFrame(forERR)
        y_oos = pd.DataFrame(y[(m1):T])
        breaks = breaks_full[(m1):T]
        
        ###############################################################
        ####FORECAST ERRORS MATRICES####
        ###for h=1
        FEEW= np.zeros((m2-window,1))
        FEGL= np.zeros((m2-window,1))
        FEFGL= np.zeros((m2-window,1))
        FETVFGL_laplacian= np.zeros((m2-window,1))
#         FETVFGL_l1= np.zeros((m2-window,1))
#         FETVFGL_grouplasso= np.zeros((m2-window,1))
#         FETVFGL_max= np.zeros((m2-window,1))
        
        #####SFEs####
        SFEEW= np.zeros((m2-window,1))
        SFEGL= np.zeros((m2-window,1))
        SFEFGL= np.zeros((m2-window,1))        
        SFETVFGL_laplacian= np.zeros((m2-window,1))
#         SFETVFGL_l1= np.zeros((m2-window,1))
#         SFETVFGL_grouplasso= np.zeros((m2-window,1))
#         SFETVFGL_max= np.zeros((m2-window,1))
        
        for j in range(0, m2-window):  
            print('T =', T, 'z =', z, 'j =', j)
            ###for h=1
            set1 = pd.DataFrame(trying)
            set1 = set1.iloc[j:(j+window),:]
    
            #####for time-varying######
            forecasters = Yhat.iloc[j:(j+window),:]
            truers = y_oos.iloc[j:(j+window),:]
            y2 = breaks[j:(j+window),].astype(int)
            y2 = y2.astype(int)
            
 
            #############################
            ###Estimating factors and loadings here
            k1=r2
            set2 = set1.to_numpy()
            set2 = StandardScaler().fit(set2).transform(set2)
            ###############################################################################
            ##########NOT time-varying for h=1 here
            ###############################################################################
            L, V = np.linalg.eigh(np.dot(set2.T, set2))
            idx = L.argsort()[::-1]
            L = L[idx]  # eigenvalues, Nx1
            V = V[:, idx]  # eigenvectors columns, NxN
            lmb = V[:, 0:k1]  # kx1
            Fhat = np.dot(set2, lmb)  # Txr (r=1 for PC1)
            Y = set1 - Fhat@lmb.T 
            Y = Y.to_numpy()
            
            covariate = Fhat
            betas = lmb.T
 ###############################################################################################################################                    
            ###for h=1
            ######## GLASSO###########
            GL = GraphicalLassoCV().fit(set1)
            theta_GL = GL.get_precision()
            
            ########Factor GLASSO#######
            FGL = GraphicalLassoCV().fit(Y)
            theta_FGL_error = FGL.get_precision()
            
            if k1==1:
                theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            else:
                theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            
            ########Time-Varying TVFGL(only precision time-varying i.e. gamma=1)#########   
            tuning_laplacian = CVlasso(penalty = '', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)##laplacian (ridge) is defauls and was used at the beginning
#             tuning_l1 = CVlasso(penalty = 'l1', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)
#             tuning_grouplasso = CVlasso(penalty = 'l2', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)
#             tuning_max = CVlasso(penalty = 'linf', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)
      
            print(tuning_laplacian.T)           
#             tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning[0][0], beta = tuning[1][0]).fit(Y, y2)
#             if k==1:
#                 theta_TVFGL=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
#             else:
#                 theta_TVFGL=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
           
            ###laplacian       
            tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
     
#             ###l1       
#             tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
#             if k1==1:
#                 theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
#             else:
#                 theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
#          ###############################################################   
#             ###group lasso     
#             tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
#             if k1==1:
#                 theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
#             else:
#                 theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
#          ###############################################################   
#             ###max    
#             tvfgl = TimeGraphicalLasso(psi = 'linf', max_iter=100, alpha = tuning_max[0][0], beta = tuning_max[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
#             if k1==1:
#                 theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
#             else:
#                 theta_TVFGL_max=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
#          ############################################################### 
        ###############################################################

           ###########Weights###################
            weight_EW = np.repeat(1/s,s)
            weight_GL = portfolios(set1,theta_GL)[0]
            weight_FGL = portfolios(set1,theta_FGL)[0]       
            weight_TVFGL_laplacian = portfolios(set1,theta_TVFGL_laplacian )[0]
#             weight_TVFGL_l1 = portfolios(set1,theta_TVFGL_l1)[0]
#             weight_TVFGL_grouplasso = portfolios(set1,theta_TVFGL_grouplasso)[0]
#             weight_TVFGL_max = portfolios(set1,theta_TVFGL_max)[0]
            
                      
            #########################################################################################################
           #######MODEL FORECASTS ARE HERE######
            FEEW[j]=y_oos.iloc[window +j,:] - weight_EW.T@Yhat.iloc[window +j,:]
            FEGL[j]=y_oos.iloc[window +j,:] - weight_GL.T@Yhat.iloc[window +j,:]
            FEFGL[j]=y_oos.iloc[window +j,:] - weight_FGL.T@Yhat.iloc[window +j,:]
            FETVFGL_laplacian[j]=y_oos.iloc[window +j,:] - weight_TVFGL_laplacian.T@Yhat.iloc[window +j,:] 
#             FETVFGL_l1[j]=y_oos.iloc[window +j,:] - weight_TVFGL_l1.T@Yhat.iloc[window +j,:] 
#             FETVFGL_grouplasso[j]=y_oos.iloc[window +j,:] - weight_TVFGL_grouplasso.T@Yhat.iloc[window +j,:] 
#             FETVFGL_max[j]=y_oos.iloc[window +j,:] - weight_TVFGL_max.T@Yhat.iloc[window +j,:] 
            
            #################SFEs ARE HERE##################
            ###for h=1
            SFEEW[j]=(FEEW[j])**2
            SFEGL[j]=(FEGL[j])**2
            SFEFGL[j]=(FEFGL[j])**2
            SFETVFGL_laplacian[j]=(FETVFGL_laplacian[j])**2
#             SFETVFGL_l1[j]=(FETVFGL_l1[j])**2
#             SFETVFGL_grouplasso[j]=(FETVFGL_grouplasso[j])**2
#             SFETVFGL_max[j]=(FETVFGL_max[j])**2

            ####################   
        ###for h=1
        MSFE_EW = np.mean(SFEEW)
        MSFE_GL = np.mean(SFEGL)
        MSFE_FGL = np.mean(SFEFGL)
        MSFE_TVFGL_laplacian = np.mean(SFETVFGL_laplacian)
#         MSFE_TVFGL_l1 = np.mean(SFETVFGL_l1)
#         MSFE_TVFGL_grouplasso = np.mean(SFETVFGL_grouplasso)
#         MSFE_TVFGL_max = np.mean(SFETVFGL_max)

       
        print('MSFE_EW =', MSFE_EW, 'MSFE_GL =', MSFE_GL, 'MSFE_FGL =', MSFE_FGL,'MSFE_TVFGL_laplacian =', MSFE_TVFGL_laplacian)

        ##########################################################
        ###for h=1
        error_EW[z] = MSFE_EW
        error_GL[z] = MSFE_GL
        error_FGL[z] = MSFE_FGL
        error_TVFGL_laplacian[z] = MSFE_TVFGL_laplacian
#         error_TVFGL_l1[z] = MSFE_TVFGL_l1
#         error_TVFGL_grouplasso[z] = MSFE_TVFGL_grouplasso
#         error_TVFGL_max[z] = MSFE_TVFGL_max
       
    #####################  
    
    ###for h=1
    cum_EW[count] = np.mean(error_EW)
    cum_GL[count] = np.mean(error_GL)
    cum_FGL[count] = np.mean(error_FGL)
    cum_TVFGL_laplacian[count] = np.mean(error_TVFGL_laplacian)
#     cum_TVFGL_l1[count] = np.mean(error_TVFGL_l1)
#     cum_TVFGL_grouplasso[count] = np.mean(error_TVFGL_grouplasso)
#     cum_TVFGL_max[count] = np.mean(error_TVFGL_max)
    
       
cum_h1 = np.concatenate((cum_EW,cum_GL,cum_FGL,cum_TVFGL_laplacian), axis=1)
    
savetxt('cum_h1_nobreak_c1is0.csv', cum_h1, delimiter=',')



In [ ]:
###Appendix Figure E7: TWO BREAKS IN BOTH PRECISION AND LOADINGS##################
########PARAMETERS################################################
gamma_opt_zero = 0
gamma_set = np.arange(0,1.05,0.05)  
alpha_set = np.array([0, 0.25, 0.5, 1, 10, 30]).astype(float)
beta_set = np.array([0, 0.25, 0.5, 1, 10, 30]).astype(float)
##################################################################
c1 = 0.75
c2 = 0.9
r = 5 #Factors in DGP
sigmaF = 1
sigmaX = 1
sigmaY = sigmaX
p1 = 8 #Number of AR(l) models
r2 = 3 #Estimated factors. Used to be 2
r3 =5 #Factors in modeling Forecast Errors
s = p1*(1+r2) #Forecasters
rho=0.2 #in regime 1
rho2 = 0.8 #in regime 2
phi = 0.8
N=100
reps = 500
#####################################################################
sample_size=range(400,1250,200)

#####PLACEHOLDERS
##########Matrices Placeholders######
###for h=1
cum_GL = np.zeros((len(sample_size),1))
cum_FGL = np.zeros((len(sample_size),1))
cum_EW = np.zeros((len(sample_size),1))
cum_TVFGL_laplacian = np.zeros((len(sample_size),1))
cum_TVFGL_l1 = np.zeros((len(sample_size),1))
cum_TVFGL_grouplasso = np.zeros((len(sample_size),1))
cum_TVFGL_laplacian_load = np.zeros((len(sample_size),1))
cum_TVFGL_l1_load = np.zeros((len(sample_size),1))
cum_TVFGL_grouplasso_load = np.zeros((len(sample_size),1))
count=-1
for T in sample_size:
    count = count + 1
    m2 = int(round(2*T/3)) #Forecasting observations
    m1=T-m2 
    window = int(round(3*m2/4))
    T1 = m1 + int(round(window/4))
    T2 = m1 + int(round(3*window/4))
    
    ###for h=1
    error_EW =  np.zeros((reps+1,1))
    error_GL =  np.zeros((reps+1,1))
    error_FGL =  np.zeros((reps+1,1))
    error_TVFGL_laplacian =  np.zeros((reps+1,1))
    error_TVFGL_l1 =  np.zeros((reps+1,1))
    error_TVFGL_grouplasso =  np.zeros((reps+1,1))
    error_TVFGL_laplacian_load =  np.zeros((reps+1,1))
    error_TVFGL_l1_load =  np.zeros((reps+1,1))
    error_TVFGL_grouplasso_load =  np.zeros((reps+1,1))
    
    
    ###Incorporating break in theta through break in c2####
   
    #######################################################
    for z in range(reps+1):  
       ########No break in c2###############################
        e = np.random.randn(T,1)      
        theta = np.zeros((T,1))  
        breaks_full = np.zeros((T,1))
        for w in range(T):
            if w <= T1:
                c2 = 0.3
                theta[w] = ((w+1)**c1)*(c2**(w))
                breaks_full[w,]=0
            if T1 < w <= T2:
                c2 = 0.6
                theta[w] = ((w+1)**c1)*(c2**(w))
                breaks_full[w,]=1
            if w > T2:
                c2 = 0.9
                theta[w] = ((w+1)**c1)*(c2**(w))
                breaks_full[w,]=2
                
                
        breaks_full = breaks_full.astype(int)
        breaks_full = np.ravel(breaks_full)
        

        #######################################################
        y1 = np.zeros((T,1)) 
        for g in range(T):
            y1[g] = theta[0:(g+1)].T@np.flip(e[0:(g+1)])
         
        
        ###################################################
        #creating Toeplitz matrix and its Cholesky decomposition
        ####For regime 1
        O = np.zeros((N, N))
        np.fill_diagonal(O, np.ones(N))     
        for h in range(1,N):
            np.fill_diagonal(O[h:], rho**np.repeat(h, N-h))
            np.fill_diagonal(O[:,h:], rho**np.repeat(h, N-h))
        
        Lambda = cholesky(O, lower=False)
        
        ######################################
        F = np.zeros((T,r))
        F[0,] = sigmaF*np.random.randn(1,r)
        v=np.random.randn(T,r)
        for j in range(0,r):
            for t in range(1,T):
                F[t,j] = phi*F[t-1,j]+sigmaF*v[t,j]
                
        ##################################################        
        ###############NO BREAKS##########################
        X = F@Lambda[0:r,] + sigmaX*np.random.randn(T,N)
        ##################################################
        alpha = np.random.randn(r,1)+1
        y2 = np.zeros((T,1))
        y2[1:T]= F[0:(T-1),]@alpha +0*sigmaY*np.random.randn((T-1),1)
        y=y1+y2
        
        Yhat = np.zeros((m2,s))
       
        
        for col in range(0,p1):  # Col=1 when no lags
            for k in range(0,r2+1): # Number of factors
                for row in range(0,m2):
                    xx = MakeAR(y[(row):(m1-1+row)], (col-1))[0]
                    if col==0 and k!=0:
                        xx = np.ones(((m1-1),1))

                    if k==0:
                        xx=xx
                        coef = np.linalg.inv(xx.T@xx)@xx.T@y[(row+col):(m1+row)]
                    else:
                        data = X[(row):(m1-1+row),]
                        L, V = np.linalg.eigh(np.dot(data.T, data))
                        idx = L.argsort()[::-1]
                        L = L[idx]  # eigenvalues, Nx1
                        V = V[:, idx]  # eigenvectors columns, NxN
                        lmb = V[:, 0:k]  # kx1
                        Fhat = np.dot(data, lmb)
                        if col==0:
                            Fhat = Fhat[(col):(m1 -1),]
                            xx = np.concatenate([xx,Fhat], axis=1)
                        else:
                            Fhat = Fhat[(col-1):(m1 -1),]
                            xx = np.concatenate([xx,Fhat], axis=1)

                        if col==0:
                            coef = np.linalg.inv(xx.T@xx)@xx.T@y[(row+col+1):(m1 +row)]
                        else:
                            coef = np.linalg.inv(xx.T@xx)@xx.T@y[(row+col):(m1 +row)]   
                    if col == 0: # 0 lags 
                        if k==0:
                            Yhat[row,((r2+1)*(col)+k)] = coef #if col=1 and k=0
                        else:
                            Yhat[row,((r2+1)*(col)+k)] = np.concatenate([np.array([1]), Fhat[Fhat.shape[0]-1,]], axis=0)@coef #if col=0 and k>0          
                    else:         # >=1 lags
                        if k==0:
                            Yhat[row,((r2+1)*(col)+k)] = np.concatenate([np.array([1]),np.ravel(np.flip(y[(m1 +row-col+1):(m1 +row+1)]))], axis=0)@coef                    
                            # Yhat[row,((r2+1)*(col)+k)] = np.array([1,np.flip(y[(m1 +row-col+1):(m1 +row+1)])])@coef #if col>1 and k=0
                        else:
                            Yhat[row,((r2+1)*(col)+k)] = np.concatenate([np.array([1]),np.ravel(np.flip(y[(m1 +row-col+1):(m1 +row+1)])), Fhat[Fhat.shape[0]-1,]], axis=0)@coef 
                            # USED TO BE np.ravel(np.flip(y[(m1 +row-col+1):(m1 +row+1)])) PLUS instead of MINUS for some reason
 ##################################################################################333
        
        forERR=np.full((Yhat.shape[0], Yhat.shape[1]), y[(m1):T]) - Yhat
               
        y =  pd.DataFrame(y)
        ###for h=1
        trying = forERR
        Yhat = pd.DataFrame(Yhat)
        forERR = pd.DataFrame(forERR)
        y_oos = pd.DataFrame(y[(m1):T])
        breaks = breaks_full[(m1):T]
        
        ###############################################################
        ####FORECAST ERRORS MATRICES####
        ###for h=1
        FEEW= np.zeros((m2-window,1))
        FEGL= np.zeros((m2-window,1))
        FEFGL= np.zeros((m2-window,1))
        FETVFGL_laplacian= np.zeros((m2-window,1))
        FETVFGL_l1= np.zeros((m2-window,1))
        FETVFGL_grouplasso= np.zeros((m2-window,1))
        
        FETVFGL_laplacian_load= np.zeros((m2-window,1))
        FETVFGL_l1_load= np.zeros((m2-window,1))
        FETVFGL_grouplasso_load= np.zeros((m2-window,1))
        
        #####SFEs####
        SFEEW= np.zeros((m2-window,1))
        SFEGL= np.zeros((m2-window,1))
        SFEFGL= np.zeros((m2-window,1))        
        SFETVFGL_laplacian= np.zeros((m2-window,1))
        SFETVFGL_l1= np.zeros((m2-window,1))
        SFETVFGL_grouplasso= np.zeros((m2-window,1))
        
        SFETVFGL_laplacian_load= np.zeros((m2-window,1))
        SFETVFGL_l1_load= np.zeros((m2-window,1))
        SFETVFGL_grouplasso_load= np.zeros((m2-window,1))
        
        for j in range(0, m2-window):
            print('T =', T, 'z =', z, 'out of', reps, 'j =', j, 'out of', m2-window)
            ###for h=1
            set1 = pd.DataFrame(trying)
            set1 = set1.iloc[j:(j+window),:]
            ###these r1 and r2 below are for time-varying
            err1 = pd.DataFrame(trying[0:T1,])
            err11 = pd.DataFrame(trying[T1:,])
            err2 = pd.DataFrame(trying[0:T2,])
            err22 = pd.DataFrame(trying[T2:,])

            #####for time-varying######
            forecasters = Yhat.iloc[j:(j+window),:]
            truers = y_oos.iloc[j:(j+window),:]
            y2 = breaks[j:(j+window),].astype(int)
            y2 = y2.astype(int)
            
            #############################
            ###Estimating factors and loadings here
            k1=r2
            ###for h=1    
            ###the part below w/ gamma_opt should be valid for all h's
            if T1 < (j+window) < T2 and j < T1:
                r1_cv = err1.iloc[j:T1,:]
                r11_cv = err11.iloc[0:(j+window-T1),:]
                r1_cv = r1_cv.to_numpy()
                r11_cv = r11_cv.to_numpy()
                gamma_opt = CV_gamma(gamma_set,set1, r1_cv, r11_cv, k1)
            elif (j+window) > T2 and T1 < j < T2:
                r2_cv = err2.iloc[j:T2,:]
                r22_cv = err22.iloc[0:(j+window-T2),:]
                r2_cv = r2_cv.to_numpy()
                r22_cv = r22_cv.to_numpy()
                gamma_opt = CV_gamma(gamma_set,set1, r2_cv, r22_cv, k1)
            elif (j+window) > T2 and j < T1:
                r1_cv = err1.iloc[j:T1,:]
                r11_cv = err11.iloc[0:(j+window-T1),:]
                r1_cv = r1_cv.to_numpy()
                r11_cv = r11_cv.to_numpy()
                gamma_opt1 = CV_gamma(gamma_set,set1, r1_cv, r11_cv, k1)
                r2_cv = err2.iloc[j:T2,:]
                r22_cv = err22.iloc[0:(j+window-T2),:]
                r2_cv = r2_cv.to_numpy()
                r22_cv = r22_cv.to_numpy()
                gamma_opt2 = CV_gamma(gamma_set,set1, r2_cv, r22_cv, k1)
            else:
                gamma_opt = 1
            
            print('gamma_opt =', gamma_opt)
           
            set2 = set1.to_numpy()
            set2 = StandardScaler().fit(set2).transform(set2)
            ####modified returns for time-varying loadings only!!!
            set2_load = set2.copy()
            for row in range(set2_load.shape[0]):
                for col in range(set2_load.shape[1]):
                    if row < (T1-j) and T1 < (j+window) < T2 and j < T1:
                        set2_load[row,col]=gamma_opt* set2_load[row,col] 
                    elif row < (T2-j) and (j+window) > T2 and T1 < j < T2:
                        set2_load[row,col]=gamma_opt* set2_load[row,col] 
                    elif (j+window) > T2 and j < T1:
                        if row < (T1-j):
                            set2_load[row,col]=gamma_opt1* set2_load[row,col] 
                        elif (T1-j) < row < (T2-j):
                            set2_load[row,col]=gamma_opt2* set2_load[row,col] 
            

            L_load, V_load = np.linalg.eigh(np.dot(set2_load.T, set2_load))
            idx_load = L_load.argsort()[::-1]
            L_load = L_load[idx_load]  # eigenvalues, Nx1
            V_load = V_load[:, idx_load]  # eigenvectors columns, NxN
            lmb_load = V_load[:, 0:k1]  # kx1
      
            ###According to Su (2017, JoE) if we obtain Fhat
            ###as usual they are only consistent for a rotational version
            ###hence, to get a consistent estimator use a two-stage procedure (OLS)
            Fhat_load = set2@lmb_load@np.linalg.inv(lmb_load.T@lmb_load)
            Y_load = set1 - Fhat_load@lmb_load.T ##these are the residuals
            
            covariate_load = Fhat_load
            betas_load = lmb_load.T
            
            
            ###############################################################################
            ##########NOT time-varying for h=1 here
            ###############################################################################
            L, V = np.linalg.eigh(np.dot(set2.T, set2))
            idx = L.argsort()[::-1]
            L = L[idx]  # eigenvalues, Nx1
            V = V[:, idx]  # eigenvectors columns, NxN
            lmb = V[:, 0:k1]  # kx1
            Fhat = np.dot(set2, lmb)  # Txr (r=1 for PC1)
            Y = set1 - Fhat@lmb.T 
            Y = Y.to_numpy()
            
            covariate = Fhat
            betas = lmb.T
            
 ###############################################################################################################################                    
            ###for h=1
            ######## GLASSO###########
            GL = GraphicalLassoCV().fit(set1)
            theta_GL = GL.get_precision()
            
            ########Factor GLASSO#######
            FGL = GraphicalLassoCV().fit(Y)
            theta_FGL_error = FGL.get_precision()
            
            if k1==1:
                theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            else:
                theta_FGL = theta_FGL_error - theta_FGL_error@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@theta_FGL_error@betas.T)@betas@theta_FGL_error
            
            ########Time-Varying TVFGL(only precision time-varying i.e. gamma=1)#########   
            tuning_laplacian = CVlasso(penalty = '', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)##laplacian (ridge) is defauls and was used at the beginning
            tuning_l1 = CVlasso(penalty = 'l1', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)
            tuning_grouplasso = CVlasso(penalty = 'l2', X=set1,Y=Y,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat, lmb=lmb, alpha_set=alpha_set, beta_set=beta_set)                       
            ###laplacian       
            tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian[0][0], beta = tuning_laplacian[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_laplacian=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
     
            ###l1       
            tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = tuning_l1[0][0], beta = tuning_l1[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_l1=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
         ###############################################################   
            ###group lasso     
            tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso[0][0], beta = tuning_grouplasso[1][0]).fit(Y, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( (np.cov(covariate, y=None, rowvar=False))**(-1)+ betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_grouplasso=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T@np.linalg.inv( np.linalg.inv(np.cov(covariate, y=None, rowvar=False))+betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas.T)@betas@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
         
            print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)
        ###############################################################   
    ############################################################### 
    #########Time-Varying TVFGL(BOTH precision and loadings time-varying)#########        
    ###############################################################
            tuning_laplacian_load = CVlasso(penalty = '', X=set1,Y=Y_load,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat_load, lmb=lmb_load, alpha_set=alpha_set, beta_set=beta_set)##laplacian (ridge) is defauls and was used at the beginning
            tuning_l1_load = CVlasso(penalty = 'l1', X=set1,Y=Y_load,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat_load, lmb=lmb_load, alpha_set=alpha_set, beta_set=beta_set)
            tuning_grouplasso_load = CVlasso(penalty = 'l2', X=set1,Y=Y_load,y2=y2,k1=k1, q=int(round(window/3)), window=window, forecasters = forecasters, truers = truers, Fhat = Fhat_load, lmb=lmb_load, alpha_set=alpha_set, beta_set=beta_set)
            
            print('laplacian =', tuning_laplacian.T, 'l1 =', tuning_l1.T,'grouplasso =', tuning_grouplasso.T)
            ###laplacian      
            tvfgl = TimeGraphicalLasso(max_iter=100, alpha = tuning_laplacian_load[0][0], beta = tuning_laplacian_load[1][0]).fit(Y_load, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_laplacian_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_laplacian_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]

            ###l1  
            ##sometimes tuning for l1 won't converge so added these:
            if len(tuning_l1_load[0]) == 0:
                alpha_l1_load = 0
            else:
                alpha_l1_load = tuning_l1_load[0][0]

            if len(tuning_l1_load[1]) == 0:
                beta_l1_load = 0
            else:
                beta_l1_load = tuning_l1_load[1][0]
            ##################################################
            tvfgl = TimeGraphicalLasso(psi = 'l1', max_iter=100, alpha = alpha_l1_load, beta = beta_l1_load).fit(Y_load, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_l1_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_l1_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
         ###############################################################   
            ###group lasso     
            tvfgl = TimeGraphicalLasso(psi = 'l2', max_iter=100, alpha = tuning_grouplasso_load[0][0], beta = tuning_grouplasso_load[1][0]).fit(Y_load, y2)  #{psi = 'laplacian', 'l1', 'l2', 'linf', 'node'}, default 'laplacian'
            if k1==1:
                theta_TVFGL_grouplasso_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( (np.cov(covariate_load, y=None, rowvar=False))**(-1)+ betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]  
            else:
                theta_TVFGL_grouplasso_load=tvfgl.precision_[tvfgl.precision_.shape[0]-1] - tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T@np.linalg.inv( np.linalg.inv(np.cov(covariate_load, y=None, rowvar=False))+betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]@betas_load.T)@betas_load@tvfgl.precision_[tvfgl.precision_.shape[0]-1]
         ###############################################################   
           ###########Weights###################
            weight_EW = np.repeat(1/s,s)
            weight_GL = portfolios(set1,theta_GL)[0]
            weight_FGL = portfolios(set1,theta_FGL)[0]       
            weight_TVFGL_laplacian = portfolios(set1,theta_TVFGL_laplacian )[0]
            weight_TVFGL_l1 = portfolios(set1,theta_TVFGL_l1)[0]
            weight_TVFGL_grouplasso = portfolios(set1,theta_TVFGL_grouplasso)[0]
            weight_TVFGL_laplacian_load = portfolios(set1,theta_TVFGL_laplacian_load )[0]
            weight_TVFGL_l1_load = portfolios(set1,theta_TVFGL_l1_load)[0]
            weight_TVFGL_grouplasso_load = portfolios(set1,theta_TVFGL_grouplasso_load)[0]
            
                      
            #########################################################################################################
           #######MODEL FORECASTS ARE HERE######
            FEEW[j]=y_oos.iloc[window +j,:] - weight_EW.T@Yhat.iloc[window +j,:]
            FEGL[j]=y_oos.iloc[window +j,:] - weight_GL.T@Yhat.iloc[window +j,:]
            FEFGL[j]=y_oos.iloc[window +j,:] - weight_FGL.T@Yhat.iloc[window +j,:]
            FETVFGL_laplacian[j]=y_oos.iloc[window +j,:] - weight_TVFGL_laplacian.T@Yhat.iloc[window +j,:] 
            FETVFGL_l1[j]=y_oos.iloc[window +j,:] - weight_TVFGL_l1.T@Yhat.iloc[window +j,:] 
            FETVFGL_grouplasso[j]=y_oos.iloc[window +j,:] - weight_TVFGL_grouplasso.T@Yhat.iloc[window +j,:] 
            
            FETVFGL_laplacian_load[j]=y_oos.iloc[window +j,:] - weight_TVFGL_laplacian_load.T@Yhat.iloc[window +j,:] 
            FETVFGL_l1_load[j]=y_oos.iloc[window +j,:] - weight_TVFGL_l1_load.T@Yhat.iloc[window +j,:] 
            FETVFGL_grouplasso_load[j]=y_oos.iloc[window +j,:] - weight_TVFGL_grouplasso_load.T@Yhat.iloc[window +j,:]
            
            #################SFEs ARE HERE##################
            ###for h=1
            SFEEW[j]=(FEEW[j])**2
            SFEGL[j]=(FEGL[j])**2
            SFEFGL[j]=(FEFGL[j])**2
            SFETVFGL_laplacian[j]=(FETVFGL_laplacian[j])**2
            SFETVFGL_l1[j]=(FETVFGL_l1[j])**2
            SFETVFGL_grouplasso[j]=(FETVFGL_grouplasso[j])**2
            SFETVFGL_laplacian_load[j]=(FETVFGL_laplacian_load[j])**2
            SFETVFGL_l1_load[j]=(FETVFGL_l1_load[j])**2
            SFETVFGL_grouplasso_load[j]=(FETVFGL_grouplasso_load[j])**2

            ####################   
        ###for h=1
        MSFE_EW = np.mean(SFEEW)
        MSFE_GL = np.mean(SFEGL)
        MSFE_FGL = np.mean(SFEFGL)
        MSFE_TVFGL_laplacian = np.mean(SFETVFGL_laplacian)
        MSFE_TVFGL_l1 = np.mean(SFETVFGL_l1)
        MSFE_TVFGL_grouplasso = np.mean(SFETVFGL_grouplasso)
        MSFE_TVFGL_laplacian_load = np.mean(SFETVFGL_laplacian_load)
        MSFE_TVFGL_l1_load = np.mean(SFETVFGL_l1_load)
        MSFE_TVFGL_grouplasso_load = np.mean(SFETVFGL_grouplasso_load)
       
        print('MSFE_EW =', MSFE_EW, 'MSFE_GL =', MSFE_GL, 'MSFE_FGL =', MSFE_FGL,'MSFE_TVFGL_laplacian =', MSFE_TVFGL_laplacian,
             'MSFE_TVFGL_l1 =', MSFE_TVFGL_l1, 'MSFE_TVFGL_grouplasso =', MSFE_TVFGL_grouplasso,
             'MSFE_TVFGL_laplacian_load =', MSFE_TVFGL_laplacian_load,
             'MSFE_TVFGL_l1_load =', MSFE_TVFGL_l1_load, 'MSFE_TVFGL_grouplasso_load =', MSFE_TVFGL_grouplasso_load)

        ##########################################################
        ###for h=1
        error_EW[z] = MSFE_EW
        error_GL[z] = MSFE_GL
        error_FGL[z] = MSFE_FGL
        error_TVFGL_laplacian[z] = MSFE_TVFGL_laplacian
        error_TVFGL_l1[z] = MSFE_TVFGL_l1
        error_TVFGL_grouplasso[z] = MSFE_TVFGL_grouplasso
        
        error_TVFGL_laplacian_load[z] = MSFE_TVFGL_laplacian_load
        error_TVFGL_l1_load[z] = MSFE_TVFGL_l1_load
        error_TVFGL_grouplasso_load[z] = MSFE_TVFGL_grouplasso_load

       
    #####################  
    
    ###for h=1
    cum_EW[count] = np.mean(error_EW)
    cum_GL[count] = np.mean(error_GL)
    cum_FGL[count] = np.mean(error_FGL)
    cum_TVFGL_laplacian[count] = np.mean(error_TVFGL_laplacian)
    cum_TVFGL_l1[count] = np.mean(error_TVFGL_l1)
    cum_TVFGL_grouplasso[count] = np.mean(error_TVFGL_grouplasso)
    
    cum_TVFGL_laplacian_load[count] = np.mean(error_TVFGL_laplacian_load)
    cum_TVFGL_l1_load[count] = np.mean(error_TVFGL_l1_load)
    cum_TVFGL_grouplasso_load[count] = np.mean(error_TVFGL_grouplasso_load)
    
       
cum_h1 = np.concatenate((cum_EW,cum_GL,cum_FGL,cum_TVFGL_laplacian, cum_TVFGL_l1,cum_TVFGL_grouplasso,cum_TVFGL_laplacian_load,
                         cum_TVFGL_l1_load,cum_TVFGL_grouplasso_load), axis=1)
    
savetxt('cum_h1_2breaks_c1is075.csv', cum_h1, delimiter=',')

